In [2]:
pip install scipy


   ---------------------------------------- 0.0/39.4 MB ? eta -:--:--
   -- ------------------------------------- 2.1/39.4 MB 11.3 MB/s eta 0:00:04
   --- ------------------------------------ 3.9/39.4 MB 10.5 MB/s eta 0:00:04
   ------ --------------------------------- 6.3/39.4 MB 10.4 MB/s eta 0:00:04
   -------- ------------------------------- 8.1/39.4 MB 10.0 MB/s eta 0:00:04
   --------- ------------------------------ 9.7/39.4 MB 9.6 MB/s eta 0:00:04
   ----------- ---------------------------- 11.8/39.4 MB 9.7 MB/s eta 0:00:03
   -------------- ------------------------- 13.9/39.4 MB 9.6 MB/s eta 0:00:03
   ---------------- ----------------------- 16.5/39.4 MB 9.9 MB/s eta 0:00:03
   ------------------- -------------------- 18.9/39.4 MB 10.1 MB/s eta 0:00:03
   --------------------- ------------------ 21.2/39.4 MB 10.3 MB/s eta 0:00:02
   ----------------------- ---------------- 23.1/39.4 MB 10.2 MB/s eta 0:00:02
   ------------------------- -------------- 24.6/39.4 MB 10.0 MB/s eta

In [3]:
import pandas as pd
import numpy as np
import os

# ==============================================================================
# 1. 환경 설정
# ==============================================================================
PATH_SCORE = './FlowScore/'
PATH_POINT = './FlowPoint/'

def load_csv(folder, filename):
    path = os.path.join(folder, filename)
    if os.path.exists(path):
        try:
            df = pd.read_csv(path)
            df.columns = [c.strip() for c in df.columns]
            return df
        except: return pd.DataFrame()
    return pd.DataFrame()

print("🚀 통합 데이터 인벤토리 구축 시작...\n")

# ==============================================================================
# 2. 데이터 로드 (모든 가용 데이터)
# ==============================================================================
# [FlowScore]
df_comp = load_csv(PATH_SCORE, 'company.csv')
df_fin = load_csv(PATH_SCORE, 'company_financial_statement.csv')
df_inc = load_csv(PATH_SCORE, 'company_income_statement.csv')
df_emp = load_csv(PATH_SCORE, 'company_employee_statistics.csv')
df_hist = load_csv(PATH_SCORE, 'company_history.csv')

# [FlowPoint]
df_trans = load_csv(PATH_POINT, 'receivable_transactions.csv')
df_receiv = load_csv(PATH_POINT, 'receivables.csv')
df_ceo = load_csv(PATH_POINT, 'pay_bnpl_customer_infos_b2c.csv')
df_css = load_csv(PATH_POINT, 'business_css_infos.csv')
df_reqs = load_csv(PATH_POINT, 'pay_bnpl_requests.csv') # 매핑용

# ==============================================================================
# 3. 기업별 데이터 보유 현황 집계 (Inventory)
# ==============================================================================
# 마스터 기업 리스트 생성
master_ids = set()
if not df_comp.empty: master_ids.update(df_comp['id'].unique())
if not df_fin.empty: master_ids.update(df_fin['company_id'].unique())
if not df_trans.empty: 
    col = 'company_id' if 'company_id' in df_trans.columns else 'company_id_tx'
    if col in df_trans.columns: master_ids.update(df_trans[col].unique())

print(f"🏢 총 식별된 기업 수: {len(master_ids)}개")

inventory = []

for cid in master_ids:
    row = {'Company_ID': cid}
    
    # 1. 재무 데이터 (FlowScore)
    row['Has_BS'] = len(df_fin[df_fin['company_id'] == cid]) if not df_fin.empty else 0
    row['Has_IS'] = len(df_inc[df_inc['company_id'] == cid]) if not df_inc.empty else 0
    row['Has_Emp'] = len(df_emp[df_emp['company_id'] == cid]) if not df_emp.empty else 0
    
    # 2. 활동 데이터 (FlowPoint)
    # 거래 내역
    tx_cnt = 0
    if not df_trans.empty:
        col = 'company_id' if 'company_id' in df_trans.columns else 'company_id_tx'
        if col in df_trans.columns:
            tx_cnt = len(df_trans[df_trans[col] == cid])
    row['Has_Trans'] = tx_cnt
    
    # 외부 신용 등급
    css_cnt = 0
    if not df_css.empty and 'company_id' in df_css.columns:
        css_cnt = len(df_css[df_css['company_id'] == cid])
    row['Has_Ext_Credit'] = css_cnt
    
    # CEO 신용 (Requests 테이블을 통해 매핑 시도 필요할 수 있음)
    # 여기서는 간단히 직접 매핑 확인
    ceo_cnt = 0
    if not df_ceo.empty and 'company_id' in df_ceo.columns:
        ceo_cnt = len(df_ceo[df_ceo['company_id'] == cid])
    row['Has_CEO_Credit'] = ceo_cnt

    # 기업명 매핑
    comp_name = f"ID_{cid}"
    if not df_comp.empty:
        match = df_comp[df_comp['id'] == cid]['company_name']
        if not match.empty: comp_name = match.values[0]
    row['Company_Name'] = comp_name
    
    inventory.append(row)

df_inv = pd.DataFrame(inventory)

# ==============================================================================
# 4. 결과 요약 및 저장
# ==============================================================================
# 데이터 등급 분류 (Tier)
def classify_tier(r):
    has_fin = (r['Has_BS'] > 0) and (r['Has_IS'] > 0)
    has_act = (r['Has_Trans'] > 0)
    
    if has_fin and has_act: return 'Tier 1 (All)'
    elif has_fin: return 'Tier 2 (Finance Only)'
    elif has_act: return 'Tier 3 (Activity Only)'
    else: return 'Tier 4 (No Data)'

df_inv['Data_Tier'] = df_inv.apply(classify_tier, axis=1)

print("\n📊 [데이터 보유 현황 요약]")
print(df_inv['Data_Tier'].value_counts())

print("\n🔍 [세부 데이터 보유율]")
print(f" - 재무제표 보유: {(df_inv['Has_BS']>0).mean()*100:.1f}%")
print(f" - 거래내역 보유: {(df_inv['Has_Trans']>0).mean()*100:.1f}%")
print(f" - 외부신용 보유: {(df_inv['Has_Ext_Credit']>0).mean()*100:.1f}%")

# 저장
df_inv.to_csv('Company_Data_Inventory.csv', index=False, encoding='utf-8-sig')
print("\n✅ 인벤토리 파일 저장 완료: Company_Data_Inventory.csv")

🚀 통합 데이터 인벤토리 구축 시작...

🏢 총 식별된 기업 수: 788개

📊 [데이터 보유 현황 요약]
Data_Tier
Tier 2 (Finance Only)     506
Tier 4 (No Data)          157
Tier 1 (All)               75
Tier 3 (Activity Only)     50
Name: count, dtype: int64

🔍 [세부 데이터 보유율]
 - 재무제표 보유: 73.7%
 - 거래내역 보유: 15.9%
 - 외부신용 보유: 61.3%

✅ 인벤토리 파일 저장 완료: Company_Data_Inventory.csv


In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# 1. 파일 로드
df = pd.read_csv('FlowScore_3.0_Monitoring_Report.csv')
df['Risk_Signals'] = df['Risk_Signals'].fillna('-')

# 2. 타겟 변수 설정 (Y)
# Danger(위험) 또는 Warning(주의) 등급을 '부실 징후 그룹(1)'으로 정의
# Normal(정상) 또는 Watch(관찰) 등급을 '안전 그룹(0)'으로 정의
df['Is_Risky'] = df['Grade'].apply(lambda x: 1 if x in ['Danger', 'Warning'] else 0)

# 3. 리스크 요인 추출 (X)
# 텍스트에 포함된 키워드로 바이너리 변수 생성
df['X_Liquidity'] = df['Risk_Signals'].apply(lambda x: 1 if '유동성' in x else 0)
df['X_Debt']      = df['Risk_Signals'].apply(lambda x: 1 if '부채' in x else 0)
df['X_LatePay']   = df['Risk_Signals'].apply(lambda x: 1 if '오후결제' in x else 0)
df['X_Delay']     = df['Risk_Signals'].apply(lambda x: 1 if '결제지연' in x else 0)
df['X_Credit']    = df['Risk_Signals'].apply(lambda x: 1 if '신용' in x else 0) # 기업/CEO 신용

risk_map = {
    'X_Liquidity': '[재무] 유동성 주의',
    'X_Debt':      '[재무] 부채 과다',
    'X_LatePay':   '[활동] 오후 결제 과다',
    'X_Delay':     '[활동] 결제 지연',
    'X_Credit':    '[신용] 신용 점수 저하'
}

# 4. 통계 검정 실행
print(f"🚀 [FlowScore 3.0] 지표 변별력 검증 (총 {len(df)}개 기업)")
print(f"{'지표명':<20} | {'상관계수(Phi)':<10} | {'적중률(Hit%)':<10} | {'유의확률(P)':<10} | {'판정'}")
print("-" * 80)

for col, name in risk_map.items():
    contingency = pd.crosstab(df[col], df['Is_Risky'])
    if contingency.shape == (2, 2):
        chi2, p, dof, ex = stats.chi2_contingency(contingency)
        n = contingency.sum().sum()
        phi = np.sqrt(chi2 / n)
        
        hits = df[df[col] == 1]
        hit_rate = hits['Is_Risky'].mean() if len(hits) > 0 else 0
        
        mark = "⭐⭐⭐" if p < 0.001 else ("⭐⭐" if p < 0.05 else "❌")
        print(f"{name:<20} | {phi:.3f}      | {hit_rate*100:.1f}%      | {p:.1e}     | {mark}")
    else:
        print(f"{name:<20} | -          | -          | -          | ⚠️ 데이터 없음")

🚀 [FlowScore 3.0] 지표 변별력 검증 (총 846개 기업)
지표명                  | 상관계수(Phi)  | 적중률(Hit%)  | 유의확률(P)    | 판정
--------------------------------------------------------------------------------
[재무] 유동성 주의          | 0.457      | 50.3%      | 2.1e-40     | ⭐⭐⭐
[재무] 부채 과다           | 0.418      | 54.3%      | 4.4e-34     | ⭐⭐⭐
[활동] 오후 결제 과다        | 0.427      | 66.7%      | 2.3e-35     | ⭐⭐⭐
[활동] 결제 지연           | 0.427      | 56.1%      | 2.1e-35     | ⭐⭐⭐
[신용] 신용 점수 저하        | 0.124      | 27.7%      | 3.1e-04     | ⭐⭐⭐
